In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
import torch.nn as nn
import numpy as np
import random
import torch.utils.data as data
from os.path import join
from os import listdir
from PIL import Image
from torchvision.transforms import *
import torchvision.transforms.functional as TF
import torch.nn.functional as F


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg",".bmp"])

def load_img(filepath, ch):
    if ch == 1:
      img = Image.open(filepath)#.convert('RGB')
    elif ch == 3:
      img = Image.open(filepath).convert('RGB')
    return img

class DatasetFromFolder(data.Dataset):
    def __init__(self, data_dir, ch):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames_blur = [join(data_dir+'Train/Real_cropped/', data) for data in sorted(listdir(data_dir+'Train/Real_cropped/')) if is_image_file(data)]
        self.image_filenames_sharp = [join(data_dir+'Train/mean_cropped/', data) for data in sorted(listdir(data_dir+'Train/mean_cropped/')) if is_image_file(data)]
        self.ch = ch

    def __getitem__(self, index):
        img_blur = load_img(self.image_filenames_blur[index], self.ch)
        img_sharp = load_img(self.image_filenames_sharp[index], self.ch)
        img_blur = transforms.ToTensor()(img_blur)
        img_sharp = transforms.ToTensor()(img_sharp)


        return img_blur, img_sharp

    def __len__(self):
        return len(self.image_filenames_blur)

class DatasetFromFolderEval(data.Dataset):
    def __init__(self, data_dir, ch):
        super(DatasetFromFolderEval, self).__init__()
        self.image_filenames_blur = [join(data_dir+'Test/Real_cropped/', data) for data in sorted(listdir(data_dir+'Test/Real_cropped/')) if is_image_file(data)]
        self.image_filenames_sharp = [join(data_dir+'Test/mean_cropped/', data) for data in sorted(listdir(data_dir+'Test/mean_cropped')) if is_image_file(data)]
        self.ch = ch

    def __getitem__(self, index):

        img_blur = load_img(self.image_filenames_blur[index], self.ch)
        img_sharp = load_img(self.image_filenames_sharp[index], self.ch)
        img_blur = transforms.ToTensor()(img_blur)
        img_sharp = transforms.ToTensor()(img_sharp)

        return img_blur, img_sharp

    def __len__(self):
        return len(self.image_filenames_blur)

def get_training_set(data_dir, ch):
    return DatasetFromFolder(data_dir, ch)

def get_eval_set(data_dir, ch):
    return DatasetFromFolderEval(data_dir, ch)

In [3]:
import torch
import torch.nn as nn
import torch.nn.init as init

class Higher(nn.Module):
    def __init__(self):
        super(Higher, self).__init__()
        padding = 1
        layers = []

        layers.append(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=1, padding=0, bias=False))

        for _ in range(10):
            layers.append(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=padding, bias=False))
            layers.append(nn.GroupNorm(num_groups=16, num_channels=64))  # 수정됨
            layers.append(nn.ReLU())
        layers.append(nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, padding=padding, bias=False))
        self.cnn1 = nn.Sequential(*layers)
        self._initialize_weights()

    def forward(self, x):
        out = self.cnn1(x)
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.GroupNorm):  # 수정됨
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)








In [4]:
class Lower(nn.Module):
    def __init__(self):
        super(Lower, self).__init__()
        padding = 1
        layers = []

        layers.append(nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=padding, bias=False))
        layers.append(nn.GroupNorm(num_groups=8, num_channels=16))  # 수정됨
        layers.append(nn.ReLU())

        for _ in range(5):
            layers.append(nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=2, bias=False, dilation=2))
            layers.append(nn.ReLU())

        layers.append(nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=padding, bias=False))
        layers.append(nn.GroupNorm(num_groups=16, num_channels=32))  # 수정됨
        layers.append(nn.ReLU())

        for _ in range(5):
            layers.append(nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=2, bias=False, dilation=2))
            layers.append(nn.ReLU())

        layers.append(nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=padding, bias=False))
        layers.append(nn.GroupNorm(num_groups=8, num_channels=16))  # 수정됨
        layers.append(nn.ReLU())

        layers.append(nn.Conv2d(in_channels=16, out_channels=3, kernel_size=3, padding=padding, bias=False))

        self.cnn2 = nn.Sequential(*layers)
        self._initialize_weights()

    def forward(self, x):
        out = self.cnn2(x)
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.GroupNorm):  # 수정됨
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

In [5]:
class Total_NN(nn.Module):
    def __init__(self, Higher, Lower):
        super(Total_NN, self).__init__()

        self.Higher = Higher()
        self.Lower = Lower()
        self.conv_tot = nn.Conv2d(in_channels=6, out_channels=3, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        x1 = self.Higher(x)
        x2 = self.Lower(x)
        y = torch.cat((x1, x2), dim=1)
        result = self.conv_tot(y)+x
        return result

In [6]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True


In [6]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import time

parser = argparse.ArgumentParser(description='Total_NN for Denoising')
parser.add_argument('--batch_size', type=int, default=35, help='training batch size')
#parser.add_argument('--sigma', type=int, default=25, help='noise level')
parser.add_argument('--lr', type=float, default=1e-4, help='initial learning rate')
parser.add_argument('--start_iter', type=int, default=1, help='the starting epoch count')
parser.add_argument('--nEpochs', type=int, default=150, help='# of iter at starting learning rate')
#parser.add_argument('--patch_size', type=int, default=128, help='patch size')
parser.add_argument('--snapshots', type=int, default=10, help='save weight file cycle')

parser.add_argument('--pretrained', type=bool, default=False, help='Is pretrained')
parser.add_argument('--gpu_mode', type=bool, default=True, help='GPU mode')
parser.add_argument('--threads', type=int, default=1, help='threads')
parser.add_argument('--seed', type=int, default=123, help='random seed to use')
parser.add_argument('--gpus', type=int, default=1, help='GPU nums')

parser.add_argument('--model_type', default='Total_NN', help='model name')
#parser.add_argument('--depth', type=int, default=17, help='depth of network')
#parser.add_argument('--n_channels', type=int, default=64, help='number of feature maps')
parser.add_argument('--image_channels', type=int, default=3, help='number of channels')
parser.add_argument('--kernel_size', type=int, default=3, help='kernel size')

parser.add_argument('--data_dir', type=str, default='/content/drive/MyDrive/Colab Notebooks AI/Dataset/', help='dataset directory') #이 경로 설정은 본인 컴퓨터대로 해야합니다.
parser.add_argument('--save_folder', type=str, default='/content/drive/MyDrive/Colab Notebooks AI/denoise_result10/', help='weight file save location') #이 경로 설정은 본인 컴퓨터대로 해야합니다.
args = parser.parse_args('')

gpus_list = range(args.gpus)
cudnn.benchmark = True
print(args)

def train(epoch):
    epoch_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader):
        img_blur, img_sharp = Variable(batch[0]), Variable(batch[1])
        #print(img_blur.shape)
        #print(img_sharp.shape)

        if cuda:
            img_sharp = img_sharp.cuda(gpus_list[0])
            img_blur = img_blur.cuda(gpus_list[0])

        optimizer.zero_grad()  #gradient를 초기에 0으로 설정
        t0 = time.time()

        prediction = model(img_blur)
        #print(img_sharp.shape)
        #print(prediction.shape)
        loss = criterion(prediction, img_sharp)

        t1 = time.time()
        epoch_loss += loss.data
        loss.backward()
        optimizer.step()

        print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, iteration, len(training_data_loader), loss.data, (t1 - t0)))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))

def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print(net)
    print('Total number of parameters: %d' % num_params)

def checkpoint(epoch):
    model_out_path = args.save_folder+"epoch_{}.pth".format(epoch)
    torch.save(model.state_dict(), model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

if __name__ == '__main__':

    if not os.path.exists(args.save_folder):
        os.makedirs(args.save_folder)

    cuda = args.gpu_mode
    if cuda and not torch.cuda.is_available():
        raise Exception("No GPU found, please run without --cuda")

    torch.manual_seed(args.seed)
    if cuda:
        torch.cuda.manual_seed(args.seed)

    print('===> Loading datasets')            # dataset을 로드한다.
    train_set = get_training_set(args.data_dir, args.image_channels)
    training_data_loader = DataLoader(dataset=train_set, num_workers=args.threads, batch_size=args.batch_size, shuffle=True)

    print('===> Building model ', args.model_type)
    if args.model_type == 'Total_NN':
        model = Total_NN(Higher, Lower)
        model = torch.nn.DataParallel(model, device_ids=gpus_list)

    criterion = nn.L1Loss()

    if args.pretrained:
        model_name = args.save_folder + "epoch_{}.pth".format(args.start_iter - 1)
        if os.path.exists(model_name):
            model.load_state_dict(torch.load(model_name, map_location=lambda storage, loc: storage))
            print('Pre-trained Denoising model is loaded.')

    if cuda:
        model = model.cuda(gpus_list[0])
        criterion = criterion.cuda(gpus_list[0])

    optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.999), eps=1e-8) # gradient 조절

    for epoch in range(args.start_iter, args.nEpochs + 1):
        train(epoch)

        if (epoch) % (args.nEpochs) == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 10.0
            print('Learning rate decay: lr={}'.format(optimizer.param_groups[0]['lr']))

        if (epoch) % (args.snapshots) == 0:
            checkpoint(epoch)


Namespace(batch_size=35, lr=0.0001, start_iter=1, nEpochs=150, snapshots=10, pretrained=False, gpu_mode=True, threads=1, seed=123, gpus=1, model_type='Total_NN', image_channels=3, kernel_size=3, data_dir='/content/drive/MyDrive/Colab Notebooks AI/Dataset/', save_folder='/content/drive/MyDrive/Colab Notebooks AI/denoise_result10/')
===> Loading datasets
===> Building model  Total_NN
===> Epoch[1](0/20): Loss: 0.2558 || Timer: 1.6668 sec.
===> Epoch[1](1/20): Loss: 0.1941 || Timer: 0.0159 sec.
===> Epoch[1](2/20): Loss: 0.1706 || Timer: 0.0048 sec.
===> Epoch[1](3/20): Loss: 0.1467 || Timer: 0.0055 sec.
===> Epoch[1](4/20): Loss: 0.1353 || Timer: 0.0048 sec.
===> Epoch[1](5/20): Loss: 0.1311 || Timer: 0.0054 sec.
===> Epoch[1](6/20): Loss: 0.1210 || Timer: 0.0046 sec.
===> Epoch[1](7/20): Loss: 0.1107 || Timer: 0.0048 sec.
===> Epoch[1](8/20): Loss: 0.1037 || Timer: 0.0053 sec.
===> Epoch[1](9/20): Loss: 0.0957 || Timer: 0.0048 sec.
===> Epoch[1](10/20): Loss: 0.0906 || Timer: 0.0053 sec

In [7]:
image_counter = 0

In [8]:
from __future__ import print_function
import argparse

import os
import torch
import torch.nn as nn
import torch.optim as optim
import time
import cv2
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from functools import reduce
from math import log10

parser = argparse.ArgumentParser(description='Total_NN network for Deblurring')
parser.add_argument('--test_batch_size', type=int, default=1, help='test batch size')
#parser.add_argument('--sigma', type=int, default=25, help='noise level')
parser.add_argument('--gpu_mode', type=bool, default=True, help='GPU mode')
parser.add_argument('--threads', type=int, default=1, help='threads')
parser.add_argument('--seed', type=int, default=123, help='random seed to use')
parser.add_argument('--patch_sizeH', type=int, default=20, help='patch size')
parser.add_argument('--patch_sizeW', type=int, default=580, help='patch size')
parser.add_argument('--gpus', type=int, default=1, help='GPU nums')
parser.add_argument('--model', type=str, default="/content/drive/MyDrive/Colab Notebooks AI/denoise_result10/epoch_150.pth", help='weight file location')

parser.add_argument('--model_type', default='Total_NN', help='model name')
#parser.add_argument('--depth', type=int, default=17, help='depth of network')
#parser.add_argument('--n_channels', type=int, default=64, help='number of feature maps')
parser.add_argument('--image_channels', type=int, default=3, help='number of channels')
#parser.add_argument('--kernel_size', type=int, default=3, help='kernel size')

parser.add_argument('--data_dir', type=str, default='/content/drive/MyDrive/Colab Notebooks AI/Dataset/', help='dataset directory')
parser.add_argument('--save_folder', type=str, default='/content/drive/MyDrive/Colab Notebooks AI/deblur_figs10/', help='result file save location')
args = parser.parse_args('')

gpus_list=range(args.gpus)
print(args)

cuda = args.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(args.seed)
criterion = nn.L1Loss()

print('===> Loading datasets')
test_set = get_eval_set(args.data_dir, args.image_channels)
testing_data_loader = DataLoader(dataset=test_set, num_workers=args.threads, batch_size=args.test_batch_size, shuffle=False)

print('===> Building model')
model = Total_NN(Higher, Lower)
model = torch.nn.DataParallel(model, device_ids=gpus_list)

model.load_state_dict(torch.load(args.model, map_location=lambda storage, loc: storage))
print('Pre-trained Denoising model is loaded.')

if cuda:
    model = model.cuda(gpus_list[0])
    criterion = criterion.cuda(gpus_list[0])

def calculate_psnr(img1, img2):
    img1 , img2 = img1.cpu().numpy(), img2.cpu().numpy()
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    #max_pixel = 1.0 if img1.max() <= 1.0 else 255.0
    max_pixel = 1.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def eval():
    avg_L1 = 0
    L1_sq = 0
    average = 0
    variance = 0
    avg_psnr = 0

    model.eval()
    for batch in testing_data_loader:
        with torch.no_grad():
            #print(f"Batch length: {len(batch)}")
            #print(f"Batch content: {batch}")
            img_blur, img_sharp = Variable(batch[0]), Variable(batch[1])

        if cuda:
            img_blur = img_blur.cuda(gpus_list[0])
            img_sharp = img_sharp.cuda(gpus_list[0])

        t0 = time.time()
        with torch.no_grad():
          prediction = model(img_blur)

        L1 = criterion(prediction, img_sharp)
        psnr = calculate_psnr(prediction, img_sharp)
        avg_L1 += L1
        avg_psnr += psnr

        t1 = time.time()
        print("===> Processing: Timer: %.4f sec. L1 : %.4f || PSNR : %.3f" % ((t1 - t0), L1, psnr))
        prediction = prediction.squeeze(0).cpu()
        save_img(prediction)
        average = avg_L1/len(testing_data_loader)
        average_psnr = avg_psnr / len(testing_data_loader)
    print("===> Processing Done, Average L1 : %.4f || Average PSNR : %.3f" % (average, average_psnr))



def save_img(image_tensor):
    global image_counter  # 전역 변수를 사용하기 위해 선언

    save_dir = args.save_folder

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 파일 이름 생성: 예를 들어, image_1.png, image_2.png, ...
    file_name = f"image_{image_counter}.png"
    save_path = os.path.join(save_dir, file_name)

    # 이미지 저장
    save_image(image_tensor, save_path)
    print("Image saved as {}".format(save_path))

    # 이미지 번호 증가
    image_counter += 1

'''def save_img(image_tensor):
    save_dir = args.save_folder

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    save_image(image_tensor, save_dir)
    print("Image saved as {}".format(save_dir))'''

if __name__ == '__main__':
    eval()

Namespace(test_batch_size=1, gpu_mode=True, threads=1, seed=123, patch_sizeH=20, patch_sizeW=580, gpus=1, model='/content/drive/MyDrive/Colab Notebooks AI/denoise_result10/epoch_150.pth', model_type='Total_NN', image_channels=3, data_dir='/content/drive/MyDrive/Colab Notebooks AI/Dataset/', save_folder='/content/drive/MyDrive/Colab Notebooks AI/deblur_figs10/')
===> Loading datasets
===> Building model
Pre-trained Denoising model is loaded.


<ipython-input-8-39e95bcd4c50>:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.model, map_location=lambda storage, loc: storage))


===> Processing: Timer: 0.0886 sec. L1 : 0.0070 || PSNR : 40.639
Image saved as /content/drive/MyDrive/Colab Notebooks AI/deblur_figs10/image_0.png
===> Processing: Timer: 0.0092 sec. L1 : 0.0066 || PSNR : 41.246
Image saved as /content/drive/MyDrive/Colab Notebooks AI/deblur_figs10/image_1.png
===> Processing: Timer: 0.0045 sec. L1 : 0.0061 || PSNR : 42.014
Image saved as /content/drive/MyDrive/Colab Notebooks AI/deblur_figs10/image_2.png
===> Processing: Timer: 0.0067 sec. L1 : 0.0066 || PSNR : 41.352
Image saved as /content/drive/MyDrive/Colab Notebooks AI/deblur_figs10/image_3.png
===> Processing: Timer: 0.0069 sec. L1 : 0.0067 || PSNR : 41.228
Image saved as /content/drive/MyDrive/Colab Notebooks AI/deblur_figs10/image_4.png
===> Processing: Timer: 0.0067 sec. L1 : 0.0065 || PSNR : 41.394
Image saved as /content/drive/MyDrive/Colab Notebooks AI/deblur_figs10/image_5.png
===> Processing: Timer: 0.0069 sec. L1 : 0.0065 || PSNR : 41.479
Image saved as /content/drive/MyDrive/Colab Not

In [26]:
avg_L1 = 0
L1_sq = 0
average = 0
variance = 0
avg_psnr = 0


for batch in testing_data_loader:
        with torch.no_grad():
            #print(f"Batch length: {len(batch)}")
            #print(f"Batch content: {batch}")
            img_blur, img_sharp = Variable(batch[0]), Variable(batch[1])

        if cuda:
            img_blur = img_blur.cuda(gpus_list[0])
            img_sharp = img_sharp.cuda(gpus_list[0])

        t0 = time.time()

        L1 = criterion(img_blur, img_sharp)
        psnr = calculate_psnr(img_blur, img_sharp)
        avg_L1 += L1
        avg_psnr += psnr

        t1 = time.time()
        print("===> Processing: Timer: %.4f sec. L1 : %.4f || PSNR : %.3f" % ((t1 - t0), L1, psnr))

        average = avg_L1/len(testing_data_loader)
        average_psnr = avg_psnr / len(testing_data_loader)
print("===> Processing Done, Average L1 : %.4f || Average PSNR : %.3f" % (average, average_psnr))

===> Processing: Timer: 0.0024 sec. L1 : 0.0118 || PSNR : 36.126
===> Processing: Timer: 0.0008 sec. L1 : 0.0141 || PSNR : 34.871
===> Processing: Timer: 0.0007 sec. L1 : 0.0098 || PSNR : 36.971
===> Processing: Timer: 0.0007 sec. L1 : 0.0108 || PSNR : 36.178
===> Processing: Timer: 0.0008 sec. L1 : 0.0145 || PSNR : 34.692
===> Processing: Timer: 0.0006 sec. L1 : 0.0141 || PSNR : 34.886
===> Processing: Timer: 0.0007 sec. L1 : 0.0135 || PSNR : 35.218
===> Processing: Timer: 0.0008 sec. L1 : 0.0147 || PSNR : 34.030
===> Processing: Timer: 0.0007 sec. L1 : 0.0147 || PSNR : 33.964
===> Processing: Timer: 0.0007 sec. L1 : 0.0173 || PSNR : 31.928
===> Processing: Timer: 0.0007 sec. L1 : 0.0144 || PSNR : 33.714
===> Processing: Timer: 0.0006 sec. L1 : 0.0168 || PSNR : 32.879
===> Processing: Timer: 0.0007 sec. L1 : 0.0144 || PSNR : 34.749
===> Processing: Timer: 0.0007 sec. L1 : 0.0146 || PSNR : 34.596
===> Processing: Timer: 0.0008 sec. L1 : 0.0150 || PSNR : 34.345
===> Processing: Timer: 0